In [2]:
import numpy as np
import pandas as pd
from pandas import DataFrame,Series

from keras import models
from keras.models import Sequential
from keras.models import Model
from keras.models import load_model
from keras import backend as K

from keras import layers
from keras.layers import Layer
from keras.layers import Input,Dense,Flatten,Embedding,Permute,Dot,Reshape
from keras.layers.convolutional import Conv1D,MaxPooling1D,MaxPooling2D
from keras.layers import Dropout
from keras.layers import LSTM,GRU

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

from keras.preprocessing import sequence
from keras.utils import np_utils

import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

import cv2

import os

C:\Users\never\Anaconda3\envs\tensorflow35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## 데이터셋 로드

In [5]:
answer = pd.read_csv(".\\train.csv",engine="python",encoding = "utf-8")

In [29]:
df = DataFrame()

df["ImageID"] = answer["ImageId_ClassId"].apply(lambda x : x.split("_")[0])
df["ClassID"] = answer["ImageId_ClassId"].apply(lambda x : x.split("_")[1])
df["EncodedPixels"] = answer["EncodedPixels"]
df['ClassId_EncodedPixels'] = df.apply(lambda row: (row['ClassID'], row['EncodedPixels']), axis = 1)
del df["EncodedPixels"]

In [31]:
grouped_EncodedPixels = df.groupby('ImageID')['ClassId_EncodedPixels'].apply(list)

In [32]:
grouped_EncodedPixels

ImageID
0002cc93b.jpg    [(1, 29102 12 29346 24 29602 24 29858 24 30114...
00031f466.jpg             [(1, nan), (2, nan), (3, nan), (4, nan)]
000418bfc.jpg             [(1, nan), (2, nan), (3, nan), (4, nan)]
000789191.jpg             [(1, nan), (2, nan), (3, nan), (4, nan)]
0007a71bf.jpg    [(1, nan), (2, nan), (3, 18661 28 18863 82 190...
000a4bcdd.jpg    [(1, 37607 3 37858 8 38108 14 38359 20 38610 2...
000f6bf48.jpg    [(1, nan), (2, nan), (3, nan), (4, 131973 1 13...
0014fce06.jpg    [(1, nan), (2, nan), (3, 229501 11 229741 33 2...
001982b08.jpg             [(1, nan), (2, nan), (3, nan), (4, nan)]
001d1b355.jpg             [(1, nan), (2, nan), (3, nan), (4, nan)]
001d3d093.jpg             [(1, nan), (2, nan), (3, nan), (4, nan)]
0025bde0c.jpg    [(1, nan), (2, nan), (3, 8458 14 8707 35 8963 ...
002af848d.jpg    [(1, nan), (2, nan), (3, nan), (4, 290800 6 29...
002e73b3c.jpg             [(1, nan), (2, nan), (3, nan), (4, nan)]
002fc4e19.jpg    [(1, 146021 3 146275 10 146529 40 146

In [85]:
file_path = ".\\train"
train_list = os.listdir(file_path)

In [86]:
img = cv2.imread(file_path +'\\'+ train_list[1],0)

In [9]:
total_set = dict()

for i in train_list[1:]:
    temp = cv2.imread(file_path + "\\" + i,0)
    total_set[i] = temp.copy()

AttributeError: 'NoneType' object has no attribute 'copy'

In [110]:
np.shape(total_set[list(total_set.keys())[1]])

(256, 1600)

## 단순 컨볼루션

In [164]:
inputs = layers.Input(shape=(1,256,1600),name = "Feed_Sentence")

conv_1 = layers.Conv2D(256,(3,3),padding="valid",activation = "relu",data_format = "channels_first")(inputs)
conv_1 = layers.Conv2D(256,(3,3),padding="valid",activation = "relu",data_format = "channels_first")(conv_1)

MaxPooling_layer = layers.MaxPooling2D((2,2))(conv_1)
Flatten_layer = layers.Flatten()(MaxPooling_layer)

outputs = layers.Dense(8,activation = "relu")(Flatten_layer)

models = Model(inputs = inputs,outputs = outputs)

In [165]:
models.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Feed_Sentence (InputLayer)   (None, 1, 256, 1600)      0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 256, 254, 1598)    2560      
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 256, 252, 1596)    590080    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 128, 126, 1596)    0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 25740288)          0         
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 205922312 
Total params: 206,514,952
Trainable params: 206,514,952
Non-trainable params: 0
______________________________________________________________

In [ ]:
SVG(model_to_dot(conv_1,show_shapes = True)).create(prog="dot",format="svg")